# Portfolio Construction using Black-Litterman Model

## Asset classes selection

| Index | Name                                  | Ticker | Variable     | Factor            |
|-------|---------------------------------------|--------|--------------|-------------------|
| 0     | SPDR S&P 500 ETF Trust                | SPY    | sp500        | Benchmark         |
| 1     | Berkshire Hathaway Inc.               | BRK.B  | eq_brkb      | Equity Value      |
| 2     | UnitedHealth Group Incorporated       | UNH    | eq_unh       | Equity Value      |
| 3     | JPMorgan Chase & Co.                  | JPM    | eq_jpm       | Equity Value      |
| 4     | Exxon Mobil Corporation               | XOM    | eq_xom       | Equity Value      |
| 5     | Broadcom Inc.                         | AVGO   | eq_avgo      | Equity Value      |
| 6     | Meta Platforms, Inc.                  | META   | eq_meta      | Equity Growth     |
| 7     | Tesla, Inc.                           | TSLA   | eq_tsla      | Equity Growth     |
| 8     | Eli Lilly and Company                 | LLY    | eq_lly       | Equity Growth     |
| 9     | Visa Inc.                             | V      | eq_visa      | Equity Growth     |
| 10    | Mastercard Incorporated               | MA     | eq_mcard     | Equity Growth     |
| 11    | Microsoft Corporation                 | MSFT   | eq_msft      | Equity Large Cap  |
| 12    | Apple Inc.                            | AAPL   | eq_aapl      | Equity Large Cap  |
| 13    | Amazon.com, Inc.                      | AMZN   | eq_amzn      | Equity Large Cap  |
| 14    | NVIDIA Corporation                    | NVDA   | eq_nvda      | Equity Large Cap  |
| 15    | Alphabet Inc.                         | GOOGL  | eq_googl     | Equity Large Cap  |
| 16    | Targa Resources Corp.                 | TRGP   | eq_trgp      | Equity Low Cap    |
| 17    | PTC Inc.                              | PTC    | eq_ptc       | Equity Low Cap    |
| 18    | Deckers Outdoor Corporation           | DECK   | eq_deck      | Equity Low Cap    |
| 19    | Atmos Energy Corporation              | ATO    | eq_ato       | Equity Low Cap    |
| 20    | Builders FirstSource, Inc.            | BLDR   | eq_bldr      | Equity Low Cap    |
| 21    | iShares Short Treasury Bond ETF       | SHV    | bond_short   | Shorterm Bond     |
| 22    | iShares 20+ Year Treasury Bond ETF    | TLT    | bond_long    | Longterm Bond     |
| 23    | United States Oil Fund LP             | USO    | com_oil      | Commodity         |
| 24    | iShares Gold Trust                    | IAU    | com_gold     | Commodity         |
| 25    | iShares MSCI USA Momentum Factor ETF  | MTUM   | momentum     | Momentum Factor   |
| 26    | Risk-free rate - Kenneth French Data  | N/A    | rf_rate      | Risk free rate    |


In [ ]:
import pandas as pd
import math
import pandas_datareader as pdr
import statsmodels.api as sm
import yfinance as yf
import numpy as np
from numpy.linalg import multi_dot
import scipy.optimize as sco

import seaborn as sns
import plotly_express as px
import matplotlib.pyplot as plt


# Format for vector output
float_formatter = "{:.6f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

Specify the tickers and variable for the data frame

In [ ]:
tickers = [
    'SPY', 'BRK-B', 'UNH', 'JPM', 'XOM', 'AVGO', 'META', 'TSLA', 'LLY', 'V',
    'MA', 'MSFT', 'AAPL', 'AMZN', 'NVDA', 'GOOGL', 'TRGP', 'PTC', 'DECK',
    'ATO', 'BLDR', 'SHV', 'TLT', 'USO', 'IAU', 'MTUM'
]
variables = [
    "sp500","eq_brkb", "eq_unh", "eq_jpm", "eq_xom", "eq_avgo", "eq_meta", "eq_tsla", 
    "eq_lly", "eq_visa", "eq_mcard", "eq_msft", "eq_aapl", "eq_amzn", "eq_nvda", 
    "eq_googl", "eq_trgp", "eq_ptc", "eq_deck", "eq_ato", "eq_bldr", "bond_short", 
    "bond_long", "com_oil", "com_gold", "momentum"
]
StartDate = '2020-04-01'
EndDate = '2023-08-01'

Download the data and save it to the `..\data` folder

In [ ]:
downloaded_data = yf.download(tickers = tickers, start=StartDate, end=EndDate, interval='1d')
downloaded_data['Adj Close'].to_csv('../data/portfolio.csv')

In [ ]:
loaded_data = pd.read_csv('../data/portfolio.csv').set_index('Date')
daily_price = loaded_data
daily_price.columns = variables
sp500 = daily_price.sp500
daily_price.drop('sp500', axis=1, inplace=True)

Compute the daily returns

In [ ]:
daily_rets = daily_price.pct_change()[1:]
daily_rets.info()

In [ ]:
fig, ax = plt.subplots(figsize=(10 , 5))
sns.heatmap(daily_rets.corr(), annot=False, vmax=1, vmin=-1, center=0, cmap='vlag')
plt.show()

Create `high_corr` filter to identify highly correlated assets and drop them from the portfolio

In [ ]:
def high_corr(data, threshold=0.5):
    col_corr = set()
    corr_matrix = data
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
mask_high_corr = high_corr(daily_rets.corr())
mask_high_corr

In [ ]:
daily_rets_filtered = daily_rets.drop(mask_high_corr , axis=1)
daily_rets_filtered['sp500'] = sp500.pct_change()
daily_rets_filtered.describe().transpose()

Calculate cummulative returns

In [ ]:
cum_rets_filtered = (1 + daily_rets_filtered).cumprod() - 1
cum_rets_filtered

Plot the daily returns data

In [ ]:
# reset the index, moving `date` as column
daily_rets_filtered = daily_rets_filtered.reset_index()
# use `melt`
df1 = daily_rets_filtered.melt(id_vars=['Date'], var_name='ticker', value_name='daily_return')
# add one more column, showing the daily_return as percent
df1['daily_return_pct'] = df1['daily_return'] * 100
df1

In [ ]:
fig = px.line(df1, x='Date',
              y='daily_return_pct', color='ticker',
              title='Performance - Daily Simple Returns',
              labels={'daily_return_pct':'daily returns (%)'})
fig.show()

Plot cummulative daily returns 

In [ ]:
# reset the index, moving `date` as column
cum_rets_filtered = cum_rets_filtered.reset_index()
df2 = cum_rets_filtered.melt(id_vars=['Date'], var_name='ticker', value_name='cum_return')
df2['cum_return_pct'] = df2['cum_return'] * 100
df2

In [ ]:
fig = px.line(df2, x='Date',
              y='cum_return_pct', color='ticker',
              title='Performance - Daily Cumulative Returns',   
              labels={'cum_return_pct':'Daily cumulative returns (%)', })
fig.show()

---

## Ledoit-Wolf Covariance Shrinkage

Implement the shrinkage estimator to estimate the covariance matrix

In [ ]:
from sklearn.covariance import LedoitWolf

In [ ]:
daily_rets.drop(mask_high_corr , axis=1)

In [ ]:
cov_shrink = LedoitWolf().fit(daily_rets.drop(mask_high_corr , axis=1))
cov_matrix = cov_shrink.covariance_

In [ ]:
pd.DataFrame(cov_matrix)

---

## Historical Weights

Calculate market cap weight

In [ ]:
market_cap = {'BRK-B':791244000000,'UNH':482258000000, 'XOM':399477000000, 'LLY':610328000000, 'MA':402400000000, 'PTC':20563000000, 'ATO':17465000000, 'USO':1470000000}

In [ ]:
def market_weight_cal(data):
    sum = 0
    for value in data.values():
        sum += value
    for key in data.keys():
        data[key] = data[key]/sum
    return data

In [ ]:
market_weight = market_weight_cal(market_cap)
market_weight = np.fromiter(market_weight.values(), dtype=float)
market_weight.reshape(-1,1)

Download Fama French data for risk free rate and the market factor terms

In [ ]:
famafrench_model = pdr.famafrench.FamaFrenchReader('F-F_Research_Data_Factors_daily',start=StartDate, end=EndDate).read()[0]
famafrench_model = famafrench_model[1:]
famafrench_model = famafrench_model/100
famafrench_model.head()

Calculate market risk aversion

$$
\lambda = \frac{\mathbb{E}(r) - r_f}{\sigma^2}
$$

In [ ]:
risk_free_rate = famafrench_model.RF
market_rets = sp500.pct_change().dropna()
market_risk_aversion = (market_rets.mean() - risk_free_rate.mean())/(market_rets.std()**2)
market_risk_aversion

In [ ]:
(market_rets.mean())*252

Calculate market implied return vector:

$$
 \Pi = \lambda \Sigma w_{\text{mkt}} \tag{5}
$$

In [ ]:
market_implied_rets = market_risk_aversion * np.matmul(cov_matrix, market_weight)
market_implied_rets

In [ ]:
# Historical mean returns
observed_exess_returns = daily_rets.drop(mask_high_corr , axis=1).mean()
observed_exess_returns

Calculate weight allocation using the sample mean returns:

$$
 w = (\lambda \Sigma)^{-1} \mu \tag{6}
$$

In [ ]:
historical_weight = np.matmul(np.linalg.inv(market_risk_aversion * cov_matrix), observed_exess_returns)
historical_weight = historical_weight.reshape(-1,1)
historical_weight

---

## Mean-Variance optimization approach

$$
\begin{equation*}
\begin{split}
    \underset{w}{\text {argmax}}\: & \mu^T w - \frac{1}{2} \lambda w^T\Sigma w\\
    \text{s.t. } \: & {\bf 1}^T w = 1, \quad w \geq 0 
\end{split}
\end{equation*} 
$$

In [ ]:
# Import Scipy
import cvxpy as cp

# Number of assets
n_assets = 8

# Define the optimization variables
mvo_weight = cp.Variable(n_assets)
risk_aversion = market_risk_aversion

# Define constraints
mvo_constraints = [
    cp.sum(mvo_weight) == 1,
    mvo_weight >= 0
]

# Define the portfolio
mvo_ret = np.array(observed_exess_returns).reshape(-1,1).T @ mvo_weight
mvo_vol = cp.quad_form(mvo_weight, cov_matrix)

# Define the problem
mvo_problem = cp.Problem(cp.Maximize(mvo_ret - mvo_vol * risk_aversion / 2 ), mvo_constraints)

# Solve the problem
mvo_problem.solve()

# Get the optimal weights
mvo_optimal_weights = mvo_weight.value

# Print the optimal weights
print("Optimal Weights:")
print(mvo_optimal_weights.reshape(-1,1).round(4)*100)

Create a function to compute the optimal weight for different risk aversion values

In [ ]:
def mean_variance_optimization(return_vector, cov_matrix, risk_aversion):
    # Convert input to NumPy array
    if isinstance(return_vector, (pd.Series, pd.DataFrame)):
        return_vector = return_vector.values
    elif not isinstance(return_vector, np.ndarray):
        raise ValueError("Input return_vector must be a NumPy array, Pandas Series, or Pandas DataFrame")

    n_assets = len(return_vector)

    # Define the optimization variables
    mvo_weight = cp.Variable(n_assets)

    # Define constraints
    mvo_constraints = [
        cp.sum(mvo_weight) == 1,
        mvo_weight >= 0
    ]

    # Define the portfolio
    mvo_ret = np.array(return_vector).reshape(-1,1).T @ mvo_weight
    mvo_vol = cp.quad_form(mvo_weight, cov_matrix)

    # Define the objective function
    mvo_objective = cp.Maximize(mvo_ret - risk_aversion * mvo_vol / 2)

    # Define the problem
    mvo_problem = cp.Problem(mvo_objective, mvo_constraints)

    # Solve the problem
    mvo_problem.solve()

    # Get the optimal weights
    mvo_optimal_weights = mvo_weight.value

    return mvo_optimal_weights

In [ ]:
mean_variance_optimization(observed_exess_returns, cov_matrix, market_risk_aversion)

In [ ]:
# Compute portfolio annual return
observed_exess_returns*252 @ historical_weight * 100

In [ ]:
# Compute portfolio risk
historical_weight.T @ cov_matrix *252 @ historical_weight * 100

---

## Maximize Sharpe Ratio portfolio approach

$$
\begin{equation*}
    \begin{split}
        \underset{w}{\text {argmax}} \; &\frac{\mu^\top w - r_f }{\sqrt{w^\top \Sigma w}} \\\\
        \text{s.t.} \quad   &w^{\top} \mathbf{1} = 1 \\
                            &w \geq 0.    
    \end{split} 
\end{equation*}
$$

In [ ]:
# Portfolio Statistics
def portfolio_stats(weights):
    weights = np.array(weights)
    port_rets = weights.T @ np.array(observed_exess_returns) - risk_free_rate.mean()
    port_vols = np.sqrt(multi_dot([weights.T, cov_matrix, weights]))
    return np.array(port_rets/port_vols)
# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)

In [ ]:
# Specify constraints, bounds and initial weights
n_assets = len(observed_exess_returns)
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0,1) for x in range(n_assets))
initial_wts = n_assets*[1./n_assets]

In [ ]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
msr_optimal_weights = opt_sharpe.x

In [ ]:
opt_sharpe

In [ ]:
msr_optimal_weights.reshape(-1,1)*100

---

## BL Implememt

### Building the Inputs

In [ ]:
tau = 1.0
P = np.array([[1,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,-1]])
Q = np.array([0.2/252, 0.005/252]).reshape(-1,1)

In [ ]:
pd.options.display.float_format = '{:,.5f}'.format
pd.DataFrame(cov_matrix)

In [ ]:
# Compute Omega matrix
Omega = np.matmul(np.matmul(P, cov_matrix), P.T)
Omega = np.diag(np.diag(Omega))
Omega

$$
\mathbb{E}(r) = \mu_\text{BL} = \left[ (\tau\Sigma)^{-1} + P^\top \Omega^{-1} P \right]^{-1} \left[ (\tau\Sigma)^{-1} \Pi + P^\top \Omega^{-1} Q \right]
$$

In [ ]:
# Compute the first half of the equation
tauSigmainv = np.linalg.inv(tau*cov_matrix) 
first_half = np.linalg.inv(tauSigmainv +  P.T @ np.linalg.inv(Omega) @ P)

# Compute the second half of the equation
second_half = (tauSigmainv @ market_implied_rets).reshape(-1,1) + P.T @ np.linalg.inv(Omega) @ Q

# New combined return vector
combined_return_vector = first_half @ second_half

combined_return_vector

In [ ]:
# Compute updated weight
blm_weights = np.matmul(np.linalg.inv(market_risk_aversion * cov_matrix), combined_return_vector).flatten()
blm_weights.round(4)*100

Create a Black-Litterman class to compute the weight with specified inputs

In [ ]:
class BlackLittermanModel:
    def __init__(self, market_weight, tau, P, Q, cov_matrix):
        self.market_weight = market_weight
        self.risk_aversion = 4.10
        self.tau = tau
        self.P = P
        self.Q = Q
        self.cov_matrix = cov_matrix
        self._market_risk_aversion = 4.10

    def combined_return_vector(self):
        # Convert market_weight to row vector
        market_weight = self.market_weight.reshape(1, -1)

        # Compute market implied return vector
        market_implied_rets = self._market_risk_aversion * np.matmul(self.cov_matrix, market_weight.T)

        # Compute Omega matrix
        Omega = np.diag(np.diag(np.matmul(np.matmul(self.P, self.cov_matrix), self.P.T)))

        # Compute the first half of the equation
        tauSigmainv = np.linalg.inv(self.tau * self.cov_matrix)
        first_half = np.linalg.inv(tauSigmainv + self.P.T @ np.linalg.inv(Omega) @ self.P)

        # Compute the second half of the equation
        second_half = (tauSigmainv @ market_implied_rets).reshape(-1, 1) + self.P.T @ np.linalg.inv(Omega) @ self.Q

        # New combined return vector
        combined_return_vector = np.matmul(first_half, second_half)

        return combined_return_vector.reshape(-1, 1)

    def blm_weight(self):
        combined_return_vector = self.combined_return_vector()

        # Compute Black Litterman weight
        blm_weight = np.matmul(np.linalg.inv(self.risk_aversion * self.cov_matrix), combined_return_vector)

        return blm_weight

---

## Different Risk Aversion Levels

In [ ]:
# Create 3 different risk aversion level
risk_seeking = 1
risk_neutral = market_risk_aversion
risk_averse = 10

In [ ]:
# weight allocation for risk seeking investors
mean_variance_optimization(observed_exess_returns, cov_matrix, risk_seeking).round(4)*100

In [ ]:
# weight allocation for risk neutral investors
mean_variance_optimization(observed_exess_returns, cov_matrix, risk_neutral).round(4)*100

In [ ]:
# weight allocation for risk averse investors
mean_variance_optimization(observed_exess_returns, cov_matrix, risk_averse).round(4)*100

---

## Portfolios Returns and Risks Analysis

In [ ]:
def portfolio_analysis(weights, annual_rets, cov_matrix, rf_rate):
    weights = weights.reshape(-1,1)
    # Compute portfolio annual return
    ret = weights.T @ annual_rets
    ret = ret[0]
    # Compute portfolio risk
    risk = np.sqrt(multi_dot([weights.T, cov_matrix, weights]))
    risk = risk[0][0]
    # Compute portfolio Sharpe ratio
    sharpe_ratio = (ret - rf_rate)/risk
    return [ret, risk, sharpe_ratio]

In [ ]:
# Define annual params
annual_rets = observed_exess_returns*252
annual_cov = cov_matrix*252
annual_rf = risk_free_rate.mean()*252

In [ ]:
# Compute the portfolio return, risk and Sharpe ratio
max_sharpe_portfolio = portfolio_analysis(msr_optimal_weights, annual_rets, annual_cov, annual_rf)
blm_portfolio = portfolio_analysis(blm_weights, annual_rets, annual_cov, annual_rf)
max_sharpe_portfolio, blm_portfolio

In [ ]:
# Find the benchmark historical return and risk
benchmark_rets = sp500.pct_change().dropna().mean()*252
benchmark_risk = sp500.pct_change().dropna().std()*np.sqrt(252)
benchmark_sr = (benchmark_rets - annual_rf)/benchmark_risk
benchmark_rets.round(4)*100, benchmark_risk.round(4)*100, benchmark_sr

Plot the cummulative portfolio return

In [ ]:
# Create a data frame of return data to plot
portfolio_ret = pd.DataFrame()
portfolio_ret['sp500'] = sp500.pct_change().dropna()
portfolio_ret['MSR'] = [daily_rets_filtered.iloc[i][1:9] @ msr_optimal_weights for i in range(len(daily_rets_filtered))]
portfolio_ret['BLM'] = [daily_rets_filtered.iloc[i][1:9] @ blm_weights for i in range(len(daily_rets_filtered))]
portfolio_ret

In [ ]:
# Calculate cummulative returns
portfolio_cum_ret = (1 + portfolio_ret).cumprod() - 1
# reset the index, moving `date` as column
portfolio_cum_ret = portfolio_cum_ret.reset_index()
portfolio_cum_ret = portfolio_cum_ret.melt(id_vars=['Date'], var_name='symbol', value_name='cum_return')
portfolio_cum_ret['cum_return_pct'] = portfolio_cum_ret['cum_return'] * 100
portfolio_cum_ret

In [ ]:
fig = px.line(portfolio_cum_ret, x='Date',
              y='cum_return_pct', color='symbol',
              title='Portfolios - Daily Cumulative Returns',   
              labels={'cum_return_pct':'Daily cumulative returns (%)', })
fig.show()